<a href="https://colab.research.google.com/github/Suiii71/Car-Damage-Classification-Repair-Cost-Prediction-A-Machine-Learning-Approach/blob/main/MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries

In [ ]:
!pip install torch torchvision pandas pillow

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
ROOT = "/content/drive/MyDrive/Car_Damage_Project"
TRAIN_CSV = os.path.join(ROOT, "CarDD_classification_train.csv")
TEST_CSV  = os.path.join(ROOT, "CarDD_classification_test.csv")

TRAIN_IMG_DIR = os.path.join(ROOT, "train2017")
TEST_IMG_DIR  = os.path.join(ROOT, "test2017")

Dataset Class

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df.head())
print(df['label'].value_counts())
classes = sorted(train_df["label"].unique())
num_classes = len(classes)

label_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_label = {i:c for c,i in label_to_idx.items()}

train_df["label_idx"] = train_df["label"].map(label_to_idx)
test_df["label_idx"]  = test_df["label"].map(label_to_idx)

classes

['crack', 'dent', 'glass shatter', 'lamp broken', 'scratch', 'tire flat']

In [ ]:
class CarDamageDataset(Dataset):
    def __init__(self, df, folder, transform=None):
        self.df = df
        self.folder = folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.folder, row["image"])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = row["label_idx"]
        return img, label

In [ ]:
from sklearn.model_selection import train_test_split

train_df2, val_df = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df["label_idx"],
    random_state=42
)

len(train_df2), len(val_df)

(2393, 423)

In [ ]:
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor()
])

test_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = CarDamageDataset(train_df2, TRAIN_IMG_DIR, train_tf)
val_dataset   = CarDamageDataset(val_df,   TRAIN_IMG_DIR, test_tf)
test_dataset  = CarDamageDataset(test_df,  TEST_IMG_DIR, test_tf)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)
test_loader  = DataLoader(test_dataset, batch_size=32)

In [ ]:
weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array(range(num_classes)),
    y=train_df2["label_idx"].values
)

weights = torch.tensor(weights, dtype=torch.float32)
weights

tensor([3.3236, 0.4852, 1.2013, 6.7599, 0.4373, 2.6948])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=weights.to(device), label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', patience=2, factor=0.5
)

EPOCHS = 30
best_val = 0
patience = 8
stop = 0

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            _, preds = torch.max(out, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    scheduler.step(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val:
        best_val = val_acc
        stop = 0
        torch.save(model.state_dict(), "mobilenet_best.pth")
    else:
        stop += 1

    if stop >= patience:
        print("Early stopping!")
        break

print("Best Validation Accuracy =", best_val)

Epoch 1/30 | Loss: 160.4101 | Val Acc: 0.1489
Epoch 2/30 | Loss: 160.0153 | Val Acc: 0.1489
Epoch 3/30 | Loss: 160.8799 | Val Acc: 0.1442
Epoch 4/30 | Loss: 161.0709 | Val Acc: 0.1418
Epoch 5/30 | Loss: 160.4805 | Val Acc: 0.1442
Epoch 6/30 | Loss: 159.5243 | Val Acc: 0.1560
Epoch 7/30 | Loss: 161.2978 | Val Acc: 0.1489
Epoch 8/30 | Loss: 161.6950 | Val Acc: 0.1489
Early stopping!
Best Validation Accuracy = 0.38534278959810875


In [ ]:
model.load_state_dict(torch.load("mobilenet_best.pth"))
model.eval()

correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        out = model(imgs)
        _, preds = torch.max(out, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy =", correct/total)

Test Accuracy = 0.37967914438502676


In [ ]:
import pandas as pd

train_df = pd.read_csv(TRAIN_CSV)
print(train_df.head(20))
print(train_df.image.nunique(), len(train_df))

print("Unique labels:", train_df.label.unique())

         image      label
0   000001.jpg  tire flat
1   000002.jpg  tire flat
2   000003.jpg  tire flat
3   000004.jpg  tire flat
4   000005.jpg  tire flat
5   000006.jpg  tire flat
6   000007.jpg  tire flat
7   000008.jpg      crack
8   000009.jpg  tire flat
9   000010.jpg  tire flat
10  000011.jpg  tire flat
11  000014.jpg    scratch
12  000018.jpg    scratch
13  000019.jpg    scratch
14  000020.jpg    scratch
15  000021.jpg    scratch
16  000022.jpg    scratch
17  000026.jpg    scratch
18  000027.jpg    scratch
19  000028.jpg       dent
2816 2816
Unique labels: ['tire flat' 'crack' 'scratch' 'dent' 'glass shatter' 'lamp broken']


In [ ]:
# =========================================================
# 0) INSTALLS (Colab)
# =========================================================
!pip install timm torch torchvision pandas pillow scikit-learn

# =========================================================
# 1) IMPORTS
# =========================================================
import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import timm
from torchvision import transforms

from sklearn.model_selection import train_test_split

# =========================================================
# 2) PATHS (CHANGE THESE)
# =========================================================
ROOT = "/content/drive/MyDrive/Car_Damage_Project"
TRAIN_CSV = os.path.join(ROOT, "CarDD_classification_train.csv")
TEST_CSV  = os.path.join(ROOT, "CarDD_classification_test.csv")

TRAIN_IMG_DIR = os.path.join(ROOT, "train2017")
TEST_IMG_DIR  = os.path.join(ROOT, "test2017")
# =========================================================
# 3) LOAD + SHUFFLE CSV (CRITICAL FIX)
# =========================================================
df = pd.read_csv(TRAIN_CSV)

# shuffle to remove class blocks (your csv is sorted by label)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df.head(10))
print("\nLabel counts:\n", df['label'].value_counts())

# =========================================================
# 4) FILTER OUT MISSING IMAGES (SAFETY)
# =========================================================
exists_mask = df["image"].apply(lambda x: os.path.exists(os.path.join(TRAIN_IMG_DIR, x)))
missing = df.loc[~exists_mask, "image"].tolist()

print("\nMissing images:", len(missing))
if len(missing) > 0:
    print("Example missing:", missing[:10])

df = df[exists_mask].reset_index(drop=True)

# =========================================================
# 5) STRATIFIED TRAIN/VAL SPLIT (CRITICAL FIX)
# =========================================================
train_df, val_df = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    stratify=df["label"]
)

print("\nTrain split counts:\n", train_df["label"].value_counts())
print("\nVal split counts:\n", val_df["label"].value_counts())

# =========================================================
# 6) LABEL ENCODING (CONSISTENT ORDER)
# =========================================================
classes = sorted(df["label"].unique())
label_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_label = {i:c for c,i in label_to_idx.items()}
num_classes = len(classes)

train_df["label_idx"] = train_df["label"].map(label_to_idx)
val_df["label_idx"]   = val_df["label"].map(label_to_idx)

print("\nClasses:", classes)
print("num_classes:", num_classes)

# =========================================================
# 7) TRANSFORMS (STRONG + NORMALIZATION)
# =========================================================
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.3,0.3,0.3,0.1),
    transforms.RandomResizedCrop(224, scale=(0.7,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

# =========================================================
# 8) DATASET CLASS
# =========================================================
class CarDamageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["image"])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = int(row["label_idx"])
        return img, label

# =========================================================
# 9) DATALOADERS
# =========================================================
train_dataset = CarDamageDataset(train_df, TRAIN_IMG_DIR, train_transform)
val_dataset   = CarDamageDataset(val_df, TRAIN_IMG_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print("\nTrain samples:", len(train_dataset))
print("Val samples:", len(val_dataset))

# =========================================================
# 10) CLASS WEIGHTS (FIX IMBALANCE)
# =========================================================
counts = train_df["label_idx"].value_counts().sort_index().values
class_weights = 1.0 / counts
class_weights = class_weights / class_weights.sum()
weights = torch.tensor(class_weights, dtype=torch.float32)

print("\nClass weights:", weights)

# =========================================================
# 11) MODEL: MobileNetV2 (timm)
# =========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model(
    "mobilenetv2_100",
    pretrained=True,
    num_classes=num_classes
).to(device)

# =========================================================
# 12) LOSS, OPTIMIZER, SCHEDULER
# =========================================================
criterion = nn.CrossEntropyLoss(weight=weights.to(device), label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=2, factor=0.5
)

# =========================================================
# 13) TRAINING LOOP (EARLY STOPPING)
# =========================================================
def train_model(model, train_loader, val_loader, epochs=40):
    best_acc = 0.0
    patience = 6
    es_counter = 0

    for epoch in range(epochs):
        # ---- train ----
        model.train()
        total_loss = 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # ---- validate ----
        model.eval()
        correct, total = 0, 0

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(1)

                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        scheduler.step(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.3f} | Val Acc: {val_acc:.4f}")

        # ---- save best ----
        if val_acc > best_acc:
            best_acc = val_acc
            es_counter = 0
            torch.save(model.state_dict(), "best_mobilenet.pth")
        else:
            es_counter += 1

        # ---- early stop ----
        if es_counter >= patience:
            print("Early stopping!")
            break

    print("\nBest Val Accuracy =", best_acc)
    return best_acc

# =========================================================
# 14) RUN TRAINING
# =========================================================
train_model(model, train_loader, val_loader, epochs=40)

# =========================================================
# 15) FINAL VAL ACCURACY CHECK
# =========================================================
def eval_accuracy(loader, model_path="best_mobilenet.pth"):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    return acc

val_acc = eval_accuracy(val_loader)
print("\nFinal Validation Accuracy =", val_acc)

# =========================================================
# 16) OPTIONAL TEST ACCURACY (IF YOU HAVE TEST CSV)
# =========================================================
if os.path.exists(TEST_CSV):
    test_df = pd.read_csv(TEST_CSV)

    # keep only test images that exist
    exists_test = test_df["image"].apply(lambda x: os.path.exists(os.path.join(TEST_IMG_DIR, x)))
    test_df = test_df[exists_test].reset_index(drop=True)

    test_df["label_idx"] = test_df["label"].map(label_to_idx)

    test_dataset = CarDamageDataset(test_df, TEST_IMG_DIR, val_transform)
    test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    test_acc = eval_accuracy(test_loader)
    print("Final TEST Accuracy =", test_acc)
else:
    print("\nNo TEST CSV found, skipping test evaluation.")

        image          label
0  000649.jpg        scratch
1  001666.jpg           dent
2  001693.jpg  glass shatter
3  003449.jpg           dent
4  002344.jpg        scratch
5  002137.jpg        scratch
6  002367.jpg           dent
7  003837.jpg        scratch
8  003868.jpg          crack
9  000963.jpg           dent

Label counts:
 label
scratch          1073
dent              967
glass shatter     391
tire flat         174
crack             141
lamp broken        70
Name: count, dtype: int64

Missing images: 0

Train split counts:
 label
scratch          912
dent             822
glass shatter    332
tire flat        148
crack            120
lamp broken       59
Name: count, dtype: int64

Val split counts:
 label
scratch          161
dent             145
glass shatter     59
tire flat         26
crack             21
lamp broken       11
Name: count, dtype: int64

Classes: ['crack', 'dent', 'glass shatter', 'lamp broken', 'scratch', 'tire flat']
num_classes: 6

Train samples: 2393
Val 

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Epoch 1/40 | Loss: 212.413 | Val Acc: 0.3924
Epoch 2/40 | Loss: 140.693 | Val Acc: 0.4232
Epoch 3/40 | Loss: 123.269 | Val Acc: 0.4515
Epoch 4/40 | Loss: 113.105 | Val Acc: 0.4657
Epoch 5/40 | Loss: 105.618 | Val Acc: 0.5083
Epoch 6/40 | Loss: 100.641 | Val Acc: 0.5461
Epoch 7/40 | Loss: 98.911 | Val Acc: 0.5154
Epoch 8/40 | Loss: 94.705 | Val Acc: 0.5579
Epoch 9/40 | Loss: 94.335 | Val Acc: 0.5603
Epoch 10/40 | Loss: 89.481 | Val Acc: 0.5319
Epoch 11/40 | Loss: 89.268 | Val Acc: 0.5603
Epoch 12/40 | Loss: 87.554 | Val Acc: 0.5863
Epoch 13/40 | Loss: 86.348 | Val Acc: 0.6076
Epoch 14/40 | Loss: 83.239 | Val Acc: 0.6052
Epoch 15/40 | Loss: 84.190 | Val Acc: 0.6052
Epoch 16/40 | Loss: 81.958 | Val Acc: 0.6312
Epoch 17/40 | Loss: 82.072 | Val Acc: 0.6241
Epoch 18/40 | Loss: 80.601 | Val Acc: 0.6123
Epoch 19/40 | Loss: 80.912 | Val Acc: 0.6194
Epoch 20/40 | Loss: 78.288 | Val Acc: 0.6028
Epoch 21/40 | Loss: 79.224 | Val Acc: 0.6194
Epoch 22/40 | Loss: 78.135 | Val Acc: 0.6336
Epoch 23/40 |